In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:


import numpy as np
import pandas as pd
from shapely.geometry import Polygon
import geopandas as gpd
import pyproj
pyproj.datadir.set_data_dir('/Users/jm/Code/mapgpt-mvp/.conda/share/proj')
import matplotlib.pyplot as plt
import os
import sys
from dotenv import load_dotenv
import pandas as pd
import pydeck as pdk
from IPython.display import display
sys.path.append('../src')

from infrastructure.gpt4 import GPT4
from infrastructure.postgres_db import PostgresDB
from application.maps.choropleth_map import ChoroplethMap
import plotly.graph_objects as go
from infrastructure.gpt4 import generate_openai_embedding_request


load_dotenv()

db_name = os.environ.get("DB_NAME")
db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")


db = PostgresDB(db_name=db_name, db_user=db_user, db_password=db_password)
comunas = db.run_gpd_query("SELECT * FROM comuna")

# Function to extract coordinates
def get_polygon_coordinates(geom):
    if geom.type == 'Polygon':
        return [(x, y) for x, y in geom.exterior.coords]
    elif geom.type == 'MultiPolygon':
        return [[(x, y) for x, y in polygon.exterior.coords] for polygon in geom.geoms][0]
    else:
        return None

comunas = comunas.to_crs(epsg=4326)
np.random.seed(9)
comunas = comunas.assign(
    lat = comunas.geom.centroid.y,
    lon = comunas.geom.centroid.x,
    coordinates = comunas.geom.apply(lambda geom: [list(coord) for coord in geom.geoms[0].exterior.coords]),
    coordinates2 = comunas.geom.apply(get_polygon_coordinates),
    score=lambda _df: np.random.randint(0, 101, size=len(_df)),
    score1=lambda _df: np.random.randint(0, 101, size=len(_df)),
    score2=lambda _df: np.random.randint(0, 101, size=len(_df)),
    score3=lambda _df: np.random.randint(0, 101, size=len(_df)),
    # geom = comunas.geom.simplify(10, preserve_topology=True)


    
)
# comunas.geom = comunas.geom.simplify(10, preserve_topology=True)
comunas = comunas.to_crs(epsg=3857)
c = comunas.set_index("codigo").head(10)
c.geom = c.geom.simplify(10, preserve_topology=True)
d = comunas.set_index("codigo").tail(10)
d.geom = d.geom.simplify(10, preserve_topology=True)


/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/geopandas/io/sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/geopandas/io/sql.py:473: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(spatial_ref_sys_sql, con)
/var/folders/8j/bf4mjlj94czf88p79v8gb24w0000gp/T/ipykernel_73220/2151933016.py:45: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lat = comunas.geom.centroid.y,
/var/folders/8j/bf4mjlj94czf88p79v8gb24w0000gp/T/ipykern

In [20]:
# query = f"""SELECT nombre 
#        FROM candidatos 
#        ORDER BY {order_by} <=> '{embedding_str}' LIMIT 1;"""

emb_openai_small


1536

NameError: name 'emb_openai_small' is not defined

In [17]:
print()

SELECT *
FROM candidatos
WHERE nombre = 'Jorge Sharp'


In [15]:
gpt4 = GPT4()

In [ ]:
gpt4.send_batch_embedding()

# Candidatos

In [17]:
nombres = pd.read_sql("SELECT DISTINCT nombre FROM candidato", con=db.engine)["nombre"]

# Generate embeddings

In [21]:
requests = [generate_openai_embedding_request(i, text) for i, text in enumerate(nombres)]

batch_id = gpt4.send_batch_embedding(requests, "batch_embedding__candidato_nombre.jsonl")

In [36]:
batch_id

'batch_AjTQEw9RRt4KpmwTOpAVgqtt'

In [41]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.0/907.0 kB 11.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.9/278.9 kB 3.1 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [42]:
import tiktoken
enc = tiktoken.get_encoding("o200k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4o")

In [53]:
nombres.dropna

0                Juan Fidel Troncoso Sandoval
1                      Agustina Salazar Rozas
2                       Gonzalo Cataldo Tello
3        Jessica Del Carmen Oyanedel Oyanedel
4              Freddy Luis Fuentealba Sanzana
                         ...                 
41100             Alexis Rodrigo Wood Escobar
41101       Nancy Eugenia Valenzuela Sanhueza
41102         Francisco Javier Briceño Zuñiga
41103                 Marisol Turres Figueroa
41104            Carlos Antonio Videla Castro
Name: nombre, Length: 41105, dtype: object

In [58]:
nombres.dropna().apply(enc.encode).apply(len).sum() * 1e-8

np.float64(0.00281912)

In [57]:
1e-8

1e-08

In [8]:
# db.get_most_similar_levenstein("candidato", "nombre", "gabriel boric")

column = "nombre"
table = "candidato"
pd.read_sql(f"""SELECT {column} 
            FROM {table} 
            ORDER BY levenshtein({column}, %s) 
            LIMIT 1;""", con=db.engine, params=("gabriel boricc",))[column].iloc[0]

'Gabriel Boric Font'

In [39]:
gpt4.get_batch(batch_id).status

'in_progress'

In [ ]:
# content = self.client.files.content(output_file_id)
# content.write_to_file(output_path)

In [ ]:
gpt4.get_batch_result(1, "batch_embedding__candidato_nombre_result.jsonl")

# Save embeddings

In [ ]:
suffix = "emb_openai_small"
table = "candidato"
column = "nombre"
embedding_column = f"{column}_{suffix}"


embedding_str = json.dumps(text_embedding)
with db.conn.cursor() as cur:
    for nombre in nombres:
        update_query = f"UPDATE {table} SET {embedding_column} = %s WHERE nombre = %s"
        cur.execute(update_query, (value, row_id))

        # Commit the changes to the database
    db.conn.commit()

        # Close the cursor and the connection
        cur.close()
        conn.close()


In [9]:
from application.services.sql_utils import to_geospatial_query
from sqlparse import format

query = """
SELECT v.votos, c.nombre AS candidato_nombre, m.nombre 
AS mesa_nombre, cm.nombre AS comuna_nombre, 
e.nombre AS eleccion_nombre, e.fecha AS eleccion_fecha 
FROM votos_mesa AS v JOIN candidatura AS cd 
ON v.candidatura_id = cd.id JOIN candidato AS c 
ON cd.candidato_id = c.id JOIN mesa AS m ON v.mesa_id = m.id 
JOIN comuna AS cm ON m.comuna_id = cm.id 
JOIN eleccion AS e ON cd.eleccion_id = e.id 
WHERE c.nombre = 'Gabriel Boric Font' 
AND e.nombre = 'Presidencial 2021 Primera Vuelta' 
AND cm.nombre = 'Paredones'
"""

# to_geospatial_query(query, {"comuna": "geom"})

formatted_query = format(to_geospatial_query(query, {"comuna": "geom"}), reindent=True, keyword_case='upper')
print(formatted_query)

SELECT v.votos,
       c.nombre AS candidato_nombre,
       m.nombre AS mesa_nombre,
       cm.nombre AS comuna_nombre,
       e.nombre AS eleccion_nombre,
       e.fecha AS eleccion_fecha,
       geom
FROM votos_mesa AS v
JOIN candidatura AS cd ON v.candidatura_id = cd.id
JOIN candidato AS c ON cd.candidato_id = c.id
JOIN mesa AS m ON v.mesa_id = m.id
JOIN comuna AS cm ON m.comuna_id = cm.id
JOIN eleccion AS e ON cd.eleccion_id = e.id
WHERE c.nombre = 'Gabriel Boric Font'
  AND e.nombre = 'Presidencial 2021 Primera Vuelta'
  AND cm.nombre = 'Paredones'


In [7]:
from bidict import bidict
from sqlglot import parse_one, exp


parsed_query = parse_one(query)
table_alias = bidict({t.alias:t.name for t in parsed_query.find_all(exp.Table)})
table_names = [t.name for t in parsed_query.find_all(exp.Table)]
geospatial_columns =  {"comuna": "geom"}
if parsed_query is None:
    raise ValueError(f"Query {query} could not be parsed.")

for col in geospatial_columns:
    if col not in table_names:
        raise ValueError(f"Geotable {col} not found in query.")


select = parsed_query.find(exp.Select)
group_by = parsed_query.find(exp.Group)

if select is None:
    raise ValueError(f"Query {query} does not contain a SELECT clause.")

# if select contains comuna, add geom
geospatial_exp = [e for e in select.expressions if type(e) == exp.Column and table_alias[e.table] in geospatial_columns.keys()]
geospatial_exp

[]

In [ ]:
data  = db.run_gpd_query()

In [21]:
print(db.get_schema())

CREATE TABLE votos_mesa (
  id integer NOT NULL,
  mesa_id integer,
  candidatura_id integer,
  votos integer
);
CREATE TABLE mesa (
  id integer NOT NULL,
  comuna_id integer,
  nombre text,
  nombre_local text,
  nombre_circunscripcion_electoral text
);
CREATE TABLE eleccion (
  id integer NOT NULL,
  tipo_eleccion_id integer,
  fecha date,
  nombre text
);
CREATE TABLE pacto (
  id integer NOT NULL,
  nombre text,
  codigo text
);
CREATE TABLE partido (
  id integer NOT NULL,
  nombre text
);
CREATE TABLE region (
  id integer NOT NULL,
  codigo integer,
  nombre text
);
CREATE TABLE subpacto (
  id integer NOT NULL,
  nombre text
);
CREATE TABLE tipo_eleccion (
  id integer NOT NULL,
  nombre text
);
CREATE TABLE candidatura (
  id integer NOT NULL,
  candidato_id integer,
  eleccion_id integer,
  partido_id integer,
  pacto_id integer,
  subpacto_id integer,
  num_en_papeleta integer,
  independiente boolean
);
CREATE TABLE candidato (
  id integer NOT NULL,
  nombre__emb_openai_s

In [ ]:
cm = ChoroplethMap()